In [1]:
from __future__ import annotations
import xarray as xr
import numpy as np
import logging 
logging.basicConfig(level=logging.INFO)
import logging
import numpy as np
import pandas as pd
import xarray as xr
import pv_functions
import os
import glob
import os
import re
import cftime

/home/s233224/anaconda3/envs/atlite_cmip6_2025/lib/python3.13/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


['/groups/EXTREMES/cutouts/europe-1980-era5.nc', '/groups/EXTREMES/cutouts/europe-1981-era5.nc', '/groups/EXTREMES/cutouts/europe-1982-era5.nc', '/groups/EXTREMES/cutouts/europe-1983-era5.nc', '/groups/EXTREMES/cutouts/europe-1984-era5.nc', '/groups/EXTREMES/cutouts/europe-1985-era5.nc', '/groups/EXTREMES/cutouts/europe-1986-era5.nc', '/groups/EXTREMES/cutouts/europe-1987-era5.nc', '/groups/EXTREMES/cutouts/europe-1988-era5.nc', '/groups/EXTREMES/cutouts/europe-1989-era5.nc', '/groups/EXTREMES/cutouts/europe-1990-era5.nc', '/groups/EXTREMES/cutouts/europe-1991-era5.nc', '/groups/EXTREMES/cutouts/europe-1992-era5.nc', '/groups/EXTREMES/cutouts/europe-1993-era5.nc', '/groups/EXTREMES/cutouts/europe-1994-era5.nc', '/groups/EXTREMES/cutouts/europe-1995-era5.nc', '/groups/EXTREMES/cutouts/europe-1996-era5.nc', '/groups/EXTREMES/cutouts/europe-1997-era5.nc', '/groups/EXTREMES/cutouts/europe-1998-era5.nc', '/groups/EXTREMES/cutouts/europe-1999-era5.nc', '/groups/EXTREMES/cutouts/europe-2000-e

/home/s233224/anaconda3/envs/atlite_cmip6_2025/lib/python3.13/site-packages/xesmf/frontend.py:718: UserWarning: Using dimensions ('y', 'x') from data variable albedo as the horizontal dimensions for the regridding.
  warnings.warn(


In [2]:
models = ["HadGEM3-GC31-LL"]  # Test with only one model
variants = ["r1i1p1f3"]  # Corresponding variant for the model
period = ["historical"]
base_path="/groups/FutureWind/SFCRAD/"
orientation1='latitude_optimal'
trigon_model='simple'
clearsky_model='simple'
tracking=None
panel = {
    "model": "huld",  # Model type
    "name": "CSi",  # Panel name
    "source": "Huld 2010",  # Source of the model

    # Used for calculating capacity per m2
    "efficiency": 0.1,  # Efficiency of the panel

    # Panel temperature coefficients
    "c_temp_amb": 1,  # Panel temperature coefficient of ambient temperature
    "c_temp_irrad": 0.035,  # Panel temperature coefficient of irradiance (K / (W/m2))

    # Reference conditions
    "r_tamb": 293,  # Reference ambient temperature (20 degC in Kelvin)
    "r_tmod": 298,  # Reference module temperature (25 degC in Kelvin)
    "r_irradiance": 1000,  # Reference irradiance (W/m^2)

    # Fitting parameters
    "k_1": -0.017162,
    "k_2": -0.040289,
    "k_3": -0.004681,
    "k_4": 0.000148,
    "k_5": 0.000169,
    "k_6": 0.000005,

    # Inverter efficiency
    "inverter_efficiency": 0.9
}
output_dir = "/work/users/s233224/Climate-Change-Impacted-Solar-Energy-Generation/power/"

In [3]:
def collect_files(base_path, models, variants, periods):
    files_model = {}

    # Define the year ranges for each period
    year_ranges = {
        "historical": range(1980, 2015),  # 2014 included
        "ssp585": range(2065, 2100)       # 2099 included
    }
    for model, variant in zip(models, variants):
        model_files = {}
        for period in periods:
            # Construct the path
            search_path = os.path.join(base_path, model, period, variant)
            # Match files with the desired pattern
            file_pattern = os.path.join(search_path, "rsds_rsdsdiff_tas_*.nc")
            matched_files = glob.glob(file_pattern)

            # Filter files by year
            filtered_files = []
            for file_path in matched_files:
                filename = os.path.basename(file_path)
                # Extract year from filename
                match = re.search(r"(\d{4})", filename)
                if match:
                    year = int(match.group(1))
                    if year in year_ranges[period]:
                        filtered_files.append(file_path)

            model_files[period] = filtered_files
        files_model[model] = model_files

    return files_model

In [4]:
files=collect_files(base_path, models, variants, period)
for model in files:
   for period in files[model]:
      files[model][period] = [
         file for file in files[model][period] if "1988" in file  # Replace "1988" with the year you want to test
    ]
print(files)

{'HadGEM3-GC31-LL': {'historical': ['/groups/FutureWind/SFCRAD/HadGEM3-GC31-LL/historical/r1i1p1f3/rsds_rsdsdiff_tas_1988.nc']}}


In [5]:
for model, periods in files.items():
        print(f"Processing model: {model}")
        for period, file_list in periods.items():
            print(f"  Processing Period: {period}")
            for file_path in file_list:
                    print(f"    Processing file: {file_path}")
                    
                    # Prepare output directory and filenames
                    output_dir_period = os.path.join(output_dir, model, period)
                    os.makedirs(output_dir_period, exist_ok=True)  # Ensure the output directory exists
                    
                    file_name = os.path.basename(file_path)
                    # Replace "rsds_rsdsdiff_tas" with "solar_power"
                    file_name = file_name.replace("rsds_rsdsdiff_tas", "solar_power")
                    output_file = os.path.join(output_dir_period, file_name)
                    
                    # Check if the solar power file already exists
                    if os.path.exists(output_file):
                        print(f"    Skipping {file_path} as {output_file} already exists.")
                        continue
                    
                    # Prepare aggregated generation file name
                    file_name_agg = file_name + "_aggregated"
                    output_file_agg = os.path.join(output_dir_period, file_name_agg)
                    
                    # Check if the aggregated generation file already exists
                    if os.path.exists(output_file_agg):
                        print(f"    Skipping {file_path} as {output_file_agg} already exists.")
                        continue

                    # Extract the year as a string for later
                    year = int(re.search(r"(\d{4})", file_name).group(1))
                    
                    # Open the file
                    #for H models and gregorian: 
                    #ds = xr.open_dataset(file_path, engine="netcdf4", decode_times=False)
                    ds = xr.open_dataset(file_path, engine="netcdf4", decode_times=True)
                    print('File opened')

Processing model: HadGEM3-GC31-LL
  Processing Period: historical
    Processing file: /groups/FutureWind/SFCRAD/HadGEM3-GC31-LL/historical/r1i1p1f3/rsds_rsdsdiff_tas_1988.nc
File opened


In [6]:
def is_leap_year(year):
    """
    Check if a year is a leap year in the Gregorian calendar.
    
    Parameters:
        year (int): The year to check.
    
    Returns:
        bool: True if the year is a leap year, False otherwise.
    """
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

In [17]:
import numpy as np
import xarray as xr
import cftime
import pandas as pd

def transform_to_gregorian(ds, year):
    """
    Transform a dataset with a 360-day calendar to a Gregorian calendar, with handling for 3-hour intervals.
    Preserves the 30-minute offset starting at 1:30 (e.g., 1:30, 4:30, etc.).
    
    Parameters:
        ds (xarray.Dataset): The dataset with a 360-day calendar.
        year (int): The year being processed.
    
    Returns:
        xarray.Dataset: The transformed dataset with a Gregorian calendar.
    """
    
    # Step 1: Build the new time index with 3-hourly timestamps for Gregorian calendar
    new_times = []
    
    # The first timestamp starts at 1:30, not 0:30
    start_hour = 1  # Starting hour is 1
    start_minute = 30  # Starting minute is 30
    
    for month in range(1, 13):
        if month == 2:
            days = 29 if is_leap_year(year) else 28
        elif month in [4, 6, 9, 11]:
            days = 30
        else:
            days = 31
        
        for day in range(1, days + 1):
            for hour in range(start_hour, 24, 3):  # 3-hour intervals, starting at 1:30
                # Add 30 minutes offset as per the original times
                new_times.append(cftime.DatetimeGregorian(year, month, day, hour, start_minute, 0))  # 30 minutes offset

    new_time_index = xr.DataArray(new_times, dims="time")
    
    # Step 2: Interpolate or match (normalized time) for the resampling
    # Original time axis: assume it's equally spaced 360-day steps (3-hour intervals)
    old_times = np.linspace(0, 1, ds.dims["time"], endpoint=False)  # normalized time in original dataset
    new_times_norm = np.linspace(0, 1, len(new_time_index), endpoint=False)  # normalized new time
    
    # Step 3: Reassign normalized time to the dataset
    ds = ds.assign_coords(time=("time", old_times))
    
    # Step 4: Interpolate onto new normalized time
    ds_interp = ds.interp(time=new_times_norm)
    
    # Step 5: Replace the interpolated normalized time with real Gregorian dates
    ds_interp = ds_interp.assign_coords(time=new_time_index)
    
    return ds_interp

def is_leap_year(year):
    """Check if a year is a leap year."""
    return (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0))




In [8]:
print(ds['time'].values)
print(ds['time'].diff('time'))

[cftime.Datetime360Day(1988, 1, 1, 1, 30, 0, 0, has_year_zero=True)
 cftime.Datetime360Day(1988, 1, 1, 4, 30, 0, 0, has_year_zero=True)
 cftime.Datetime360Day(1988, 1, 1, 7, 30, 0, 0, has_year_zero=True) ...
 cftime.Datetime360Day(1988, 12, 30, 16, 30, 0, 0, has_year_zero=True)
 cftime.Datetime360Day(1988, 12, 30, 19, 30, 0, 0, has_year_zero=True)
 cftime.Datetime360Day(1988, 12, 30, 22, 30, 0, 0, has_year_zero=True)]
<xarray.DataArray 'time' (time: 2879)> Size: 23kB
array([10800000000000, 10800000000000, 10800000000000, ...,
       10800000000000, 10800000000000, 10800000000000],
      dtype='timedelta64[ns]')
Coordinates:
  * time     (time) object 23kB 1988-01-01 04:30:00 ... 1988-12-30 22:30:00


In [18]:
if model in ["HadGEM3-GC31-LL", "HadGEM3-GC31-MM"]:
    ds = transform_to_gregorian(ds, year)
else:
    # Decode time normally for other models
    ds = xr.decode_cf(ds)
print('Time decoded')

/tmp/ipykernel_65390/3502027886.py:43: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  old_times = np.linspace(0, 1, ds.dims["time"], endpoint=False)  # normalized time in original dataset


Time decoded


In [19]:
ds['time'].sel(time='1988-02').values

array([cftime.DatetimeGregorian(1988, 2, 1, 1, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 4, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 7, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 10, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 13, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 16, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 19, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 1, 22, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 2, 1, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 2, 4, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 2, 7, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 2, 10, 30, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(1988, 2, 2, 13

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

# Extract the variable and filter for February 1988
solar_irradiance = ds['rsds'].sel(time=slice('1988-02-01', '1988-02-28'))

# Aggregate over latitude and longitude (mean irradiance for each time step)
mean_irradiance = solar_irradiance.mean(dim=['lat', 'lon'])

# Plot time vs. mean irradiance
plt.figure(figsize=(10, 6))
plt.plot(mean_irradiance['time'], mean_irradiance, marker='o', linestyle='-', color='b')
plt.title("Mean Solar Irradiance in February 1988 (CanESM5)")
plt.xlabel("Time")
plt.ylabel("Mean Irradiance (kWh/m²)")
plt.grid(True)
plt.show()